In [177]:
from __future__ import print_function

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D, Layer
from keras.layers import Conv1D, MaxPooling1D, Embedding, Convolution1D, Merge, Dropout, Activation
from keras.models import Model, Sequential
from keras.layers.merge import concatenate, Concatenate
from load_glove_embeddings import load_glove_embeddings
import re
from keras import backend as K
from keras.engine.topology import Layer


In [73]:
BASE_DIR = ''
GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')
TEXT_DATA_DIR = os.path.join(BASE_DIR, 'news20/20_newsgroup')
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 400001
EMBEDDING_DIM = 50
VALIDATION_SPLIT = 0.2

In [68]:
word2index, embedding_matrix = load_glove_embeddings(GLOVE_DIR + '/glove.6B.50d.txt', embedding_dim=50)

In [69]:
len(word2index)

400001

In [58]:
X = []
for doc in text:
a    doc_vector = []
    seq = text_to_word_sequence(texts[0])
    for word in seq

['archive',
 'name',
 'atheism',
 'resources',
 'alt',
 'atheism',
 'archive',
 'name',
 'resources',
 'last',
 'modified',
 '11',
 'december',
 '1992',
 'version',
 '1',
 '0',
 'atheist',
 'resources',
 'addresses',
 'of',
 'atheist',
 'organizations',
 'usa',
 'freedom',
 'from',
 'religion',
 'foundation',
 'darwin',
 'fish',
 'bumper',
 'stickers',
 'and',
 'assorted',
 'other',
 'atheist',
 'paraphernalia',
 'are',
 'available',
 'from',
 'the',
 'freedom',
 'from',
 'religion',
 'foundation',
 'in',
 'the',
 'us',
 'write',
 'to',
 'ffrf',
 'p',
 'o',
 'box',
 '750',
 'madison',
 'wi',
 '53701',
 'telephone',
 '608',
 '256',
 '8900',
 'evolution',
 'designs',
 'evolution',
 'designs',
 'sell',
 'the',
 'darwin',
 'fish',
 "it's",
 'a',
 'fish',
 'symbol',
 'like',
 'the',
 'ones',
 'christians',
 'stick',
 'on',
 'their',
 'cars',
 'but',
 'with',
 'feet',
 'and',
 'the',
 'word',
 'darwin',
 'written',
 'inside',
 'the',
 'deluxe',
 'moulded',
 '3d',
 'plastic',
 'fish',
 'is',


In [ ]:
tokenizer.texts_

In [18]:
sequences = tokenizer.texts_to_sequences(texts, )

In [22]:
    word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 174074 unique tokens.


In [23]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (19997, 1000)
Shape of label tensor: (19997, 20)


In [43]:
# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

In [44]:
x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]


In [205]:
x_1 = Input(shape=(MAX_SEQUENCE_LENGTH,))
x_2 = Input(shape=(MAX_SEQUENCE_LENGTH,))
x_3 = Input(shape=(MAX_SEQUENCE_LENGTH,))

f = Embedding(MAX_NUM_WORDS,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)(x_1)
s = Embedding(MAX_NUM_WORDS,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)(x_2)
t = Embedding(MAX_NUM_WORDS,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)(x_3)


In [206]:
f = Conv1D(filters=200,
                     kernel_size = 4,
                     padding='valid',
                     activation='relu',
                     strides=1)(f)
f = GlobalMaxPooling1D()(f)

s = Conv1D(filters=200,
                     kernel_size = 4,
                     padding='valid',
                     activation='relu',
                     strides=1)(s)
s = GlobalMaxPooling1D()(s)


t = Conv1D(filters=200,
                     kernel_size = 4,
                     padding='valid',
                     activation='relu',
                     strides=1)(t)
t = GlobalMaxPooling1D()(t)

In [226]:
fs = Dense(units=10, activation='relu')(Concatenate()([f,s]))
st = Dense(units=10, activation='relu')(Concatenate()([s,t]))

In [227]:
join_layer = Concatenate()([fs, f, s, t, st])
hidden = Dense(units=400, activation='relu')(join_layer)
hidden = Dropout(0.2)(hidden)
is_coherrent = Dense(units=1, activation='sigmoid')(hidden)

In [228]:
model = Model(inputs=[x_1,x_2,x_3], outputs=is_coherrent)

In [229]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, 1000, 50)     20000050    input_8[0][0]                    
__________________________________________________________________________________________________
embedding_

In [210]:
adadelta = optimizers.Adadelta(rho=0.95, epsilon=1e-06)
# TODO: Use map_score as a metric here?
model.compile(optimizer=adadelta,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

NameError: name 'optimizers' is not defined

In [121]:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(MAX_NUM_WORDS,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [191]:
class SimilarityLayer(Layer):

    def __init__(self,dim=200, **kwargs):
        self.dim = dim
        super(SimilarityLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(self.dim, self.dim),
                                      initializer='uniform',
                                      trainable=True)
        super(SimilarityLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        return K.dot(K.dot(x[0][:self.dim], self.kernel),x[0][self.dim:])

    def compute_output_shape(self, input_shape):
        return (1)

In [126]:
def get_embedded_conv(embedding_layer,filters,kernel_size):
    conv_model = Sequential()
    conv_model.add(embedding_layer)

    # we add a Convolution1D, which will learn filters
    # word group filters of size filter_length:
    conv_model.add(Conv1D(filters=200,
                     kernel_size = 4,
                     padding='valid',
                     activation='relu',
                     strides=1))
    # we use max pooling:
    conv_model.add(GlobalMaxPooling1D())
    print(conv_model.summary())
    return conv_model

In [151]:
conv_f = get_embedded_conv(embedding_layer,250,4)
conv_s = get_embedded_conv(embedding_layer,250,4)
conv_t = get_embedded_conv(embedding_layer,250,4)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 1000, 50)          20000050  
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 997, 200)          40200     
_________________________________________________________________
global_max_pooling1d_127 (Gl (None, 200)               0         
Total params: 20,040,250
Trainable params: 40,200
Non-trainable params: 20,000,050
_________________________________________________________________
None
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 1000, 50)          20000050  
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 997, 200)          40200     
______________________________________________________

In [173]:
fs = Sequential()
fs.add(Merge([conv_f, conv_s],mode='concat'))
fs.add(Dense(10, activation='relu'))

st = Sequential()
st.add(Merge([conv_s, conv_t],mode='concat'))
st.add(Dense(10, activation='relu'))

model =Sequential()
model.add(Concatenate([fs, conv_f, conv_s, conv_t, st]))
model.add(Dense(units=400, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.summary()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  


ValueError: The first layer in a Sequential model must get an `input_shape` or `batch_input_shape` argument.

In [150]:
get_embedded_conv(embedding_layer,250,4)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 1000, 50)          20000050  
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 997, 200)          40200     
_________________________________________________________________
global_max_pooling1d_126 (Gl (None, 200)               0         
Total params: 20,040,250
Trainable params: 40,200
Non-trainable params: 20,000,050
_________________________________________________________________
None


In [120]:
ngram_filters = [2, 3, 4, 5]
conv_hidden_units = [200, 200, 200, 200]
convsM = []
for n_gram, hidden_units in zip(ngram_filters, conv_hidden_units):
    conv_ = Sequential()
    conv_.add(embedding_layer)
    conv_.add(Convolution1D(nb_filter=hidden_units,
                             filter_length=n_gram,
                             border_mode='same',
                             activation='tanh', name='Convolution-'+str(n_gram)+"gram"))
    
    conv_.add(GlobalMaxPooling1D())
    print(conv_.summary())
    convsM.append(conv_)

model = Sequential()
model.add(Merge(convsM, mode='concat'))
model.add(Dense(units=400, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.summary()
# conv =concatenate(inputs=convsM,axis=1)
# model.add( Dense(128, activation='relu')(conv))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="tanh", name="Convolution-2gram", filters=200, kernel_size=2, padding="same")`
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="tanh", name="Convolution-3gram", filters=200, kernel_size=3, padding="same")`
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="tanh", name="Convolution-4gram", filters=200, kernel_size=4, padding="same")`
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="tanh", name="Convolution-5gr

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 1000, 50)          20000050  
_________________________________________________________________
Convolution-2gram (Conv1D)   (None, 1000, 200)         20200     
_________________________________________________________________
global_max_pooling1d_117 (Gl (None, 200)               0         
Total params: 20,020,250
Trainable params: 20,200
Non-trainable params: 20,000,050
_________________________________________________________________
None
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 1000, 50)          20000050  
_________________________________________________________________
Convolution-3gram (Conv1D)   (None, 1000, 200)         30200     
______________________________________________________

In [ ]:
conv_f =

In [14]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 400, 50)           250000    
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 397, 250)          50250     
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 250)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 250)               62750     
_________________________________________________________________
dropout_3 (Dropout)          (None, 250)               0         
_________________________________________________________________
activation_5 (Activation)    (None, 250)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 251       
__________

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)


In [82]:
import json

from keras.preprocessing.sequence import pad_sequences

from model import *

DATASET_DIR = 'data/cui/processed/'


def pad_sequences_(sequences):
    ans = []
    for sequence in sequences:
        ans.extend(pad_sequences(sequence, maxlen=MAX_SEQUENCE_LENGTH))
    return ans

def pad_all(dict_):
    """

    :param dict:
    :return: 0 -> [   0 -> first sentence
                     1 -> second sentence
                     2 -> third sentence
                 ],
             1 -> label
    """
    x_1 = dict_['firsts']
    x_2 = dict_['seconds']
    x_3 = dict_['thirds']
    
#     y = []
#     for i in range(len(dict_['firsts'])):
#         y.extend([dict_['labels'][i] for j in range(len(dict_['firsts'][i]))])
    x_1 = pad_sequences(x_1, maxlen=MAX_SEQUENCE_LENGTH)
    x_2 = pad_sequences(x_2, maxlen=MAX_SEQUENCE_LENGTH)
    x_3 = pad_sequences(x_3, maxlen=MAX_SEQUENCE_LENGTH)

        
    return [[x_1, x_2, x_3],dict_['labels']]

In [83]:
# print("Loading train")
# with open(DATASET_DIR + "train.json") as f:
#     train = json.load(f)
#     print("Loaded train")
# #
# print("Loading test")
# with open(DATASET_DIR + "test.json") as f:
#     test = pad_all(json.load(f))

# print("Loading dev")

with open(DATASET_DIR + "dev.json") as f:
    dev = json.load(f)

In [84]:
len(dev['thirds'][1])

24

In [85]:
train_padded = pad_all(dev)

In [71]:
train_padded[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [14]:
x = train['firsts']
y = train['labels']

In [15]:
len(x)

20580

In [16]:
len(y)

20580

In [22]:
x[4]

[[5918, 1110],
 [1993, 4868, 2404, 1110, 1968],
 [2159, 5918, 1110],
 [1864, 4868, 3814, 2159, 1911, 4868, 5025],
 [1534, 2159, 41, 1864, 4652],
 [3880, 4868, 1911, 5140, 7, 1911, 1968],
 [2159, 4868],
 [41, 3814, 1864, 1911, 1110, 7, 1534, 1110],
 [7, 41, 1911, 1534, 3420, 1110, 1110, 1968],
 [1],
 [7, 3814, 1968],
 [2159, 5918, 1110],
 [1534, 5918, 6479, 2159, 2159, 1110, 1911],
 [1864, 4868, 3814, 2159, 41, 3814, 6479, 1110, 1968],
 [2]]

In [ ]:
model = get_model()
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

print(model.summary())